In [1]:
# import packages
import pandas as pd
import numpy as np
from sklearn.ensemble import RandomForestRegressor
from sklearn.metrics import mean_absolute_error,r2_score
import xgboost as xgb
from sklearn.model_selection import GridSearchCV
from joblib import dump, load

Adapting datasets to be interpreted

In [2]:
X_sample = pd.read_csv("data/ais_train.csv", delimiter='|')
X_sample.to_csv('data/ais_train_modified.csv', index=False)
extra_vessels = pd.read_csv("data/vessels.csv", on_bad_lines='skip', delimiter='|')
extra_vessels.to_csv('data/vessels_modified.csv', index=False)
extra_ports = pd.read_csv("data/ports.csv", on_bad_lines='skip', delimiter='|')
extra_ports.to_csv('data/ports_modified.csv', index=False)
extra_schedules = pd.read_csv("data/schedules_to_may_2024.csv", on_bad_lines='skip', delimiter='|')
extra_schedules.to_csv('data/schedules_to_may_2024_modified.csv', index=False)

In [3]:
X_evaluation = pd.read_csv("data/ais_test.csv",)
extra_ports = pd.read_csv("data/ports_modified.csv")
extra_vessels = pd.read_csv("data/vessels_modified.csv")
extra_schedules = pd.read_csv("data/schedules_to_may_2024_modified.csv")
X_original = pd.read_csv("data/ais_train_modified.csv")
X_original['etaRaw'] = pd.to_datetime('2024-' + X_original['etaRaw']+ ':00', format='%Y-%m-%d %H:%M:%S',errors='coerce')
X_original = X_original.dropna(subset=['etaRaw'])

Changing current data into previous data for the train file

In [4]:
def past_course(original):
    original=original.reset_index()

    original['prev_lat'] = original['latitude'].shift(1).fillna(original['latitude'].iloc[0])
    original['prev_lon'] = original['longitude'].shift(1).fillna(original['longitude'].iloc[0])
    original['time_2'] = original['time'].shift(1).fillna(original['time'].iloc[0])
    original['cog'] = original['cog'].shift(1)
    original['sog'] = original['sog'].shift(1)
    original['rot'] = original['rot'].shift(1)
    original['heading'] = original['heading'].shift(1)
    original['navstat'] = original['navstat'].shift(1)
    original.loc[0,['cog','sog','rot','heading','navstat']]=[0,0,0,0,0]

    return original

def adapting_training_data(original):
    ships = original['vesselId'].unique()
    new = original.copy()
    new['time'] = pd.to_datetime(new['time'])
    new['prev_lat']=original['latitude']
    new['prev_lon']=original['longitude']
    new['time_2'] = new['time']
    new = new.reset_index()
    final = pd.DataFrame(columns=new.columns)
    for c in ships:
        one_ship = new[new['vesselId'] == c].copy()
        new_filtered = past_course(one_ship)
        final = pd.concat([final, new_filtered], ignore_index=True)
    final = final.sort_values(by='index')
    final = final.drop(['index'],axis=1)
    final = final.reset_index(drop=True)
    final = final.drop(['level_0'],axis=1)
    final['time_dif'] = (final['time']-final['time_2']).dt.total_seconds()/3600
    return(final)

def adapting_test_data (evaluation,training):
    evalu=evaluation.copy()
    evalu['time'] = pd.to_datetime(evalu['time'])
    evalu['time_2'] = evalu['time']
    evalu[['cog','sog','heading','navstat','latitude','longitude']] = 0.1
    evalu['etaRaw'] = evalu['time']
    evalu['portId'] ='1'

    train=training.copy()
    train['time'] = pd.to_datetime(train['time'])
    train['time_2'] = train['time']
    evalu = evalu.drop(['ID','scaling_factor'],axis=1)

    final = pd.concat([train, evalu], ignore_index=True)
    x = adapting_training_data(final)
    ships = x['vesselId'].unique()
    last_one = pd.DataFrame(columns=x.columns)
    for c in ships:
        one_ship = x[x['vesselId'] == c].copy()
        one_ship=one_ship.reset_index()
        one_ship['etaRaw'] = one_ship['etaRaw'].shift(1)
        one_ship['portId'] = one_ship['portId'].shift(1)
        last_one = pd.concat([last_one, one_ship], ignore_index=True)
    last_one = last_one.sort_values(by='index')
    last_one = last_one.drop(['index'],axis=1)
    last_one = last_one.reset_index(drop=True)
    evalu = last_one.iloc[len(train):]
    ships = evalu['vesselId'].unique()
    final = final.iloc[0:0]
    for c in ships:
        filtered = evalu[evalu['vesselId'] == c].copy()
        filtered = filtered.reset_index()
        filtered[['time_2','cog','sog','rot','heading','navstat','etaRaw','prev_lat','prev_lon','portId']] = filtered.iloc[0][['time_2','cog','sog','rot','heading','navstat','etaRaw','prev_lat','prev_lon','portId']]
        final = pd.concat([final,filtered], ignore_index=True)
    
    final['time_dif'] = (final['time']-final['time_2']).dt.total_seconds()/3600
    final = final.sort_values(by='index')
    final = final.drop(['index'],axis=1)
    final = final.reset_index(drop=True)
    return final

In [5]:
test =  adapting_test_data(X_evaluation,X_original)
train = adapting_training_data(X_original)

C:\Users\jtlas\AppData\Local\Temp\ipykernel_26908\3899742820.py:28: FutureWarning: The behavior of DataFrame concatenation with empty or all-NA entries is deprecated. In a future version, this will no longer exclude empty or all-NA columns when determining the result dtypes. To retain the old behavior, exclude the relevant entries before the concat operation.
  final = pd.concat([final, new_filtered], ignore_index=True)
C:\Users\jtlas\AppData\Local\Temp\ipykernel_26908\3899742820.py:58: FutureWarning: The behavior of DataFrame concatenation with empty or all-NA entries is deprecated. In a future version, this will no longer exclude empty or all-NA columns when determining the result dtypes. To retain the old behavior, exclude the relevant entries before the concat operation.
  last_one = pd.concat([last_one, one_ship], ignore_index=True)
C:\Users\jtlas\AppData\Local\Temp\ipykernel_26908\3899742820.py:58: FutureWarning: The behavior of DataFrame concatenation with empty or all-NA entrie

In [10]:
train['time_to_destiny'] = (train['etaRaw']-train['time']).dt.total_seconds()/3600
test['time_to_destiny'] = (test['etaRaw']-test['time']).dt.total_seconds()/3600
train['Sailing'] = np.where( train['navstat'] == 0, 1, 0)
test['Sailing'] = np.where( test['navstat'] == 0, 1, 0)
extra_ports.rename(columns={'latitude': 'port_lat','longitude': 'port_lon'}, inplace=True)
extra_ports = extra_ports.drop(['name','portLocation','UN_LOCODE','countryName','ISO'],axis=1)
train = pd.merge(train, extra_ports, on='portId', how='inner')
test = pd.merge(test, extra_ports, on='portId', how='inner')

In [14]:
test_full = test.copy()
test_full = test_full.drop(['cog','sog','rot','heading','navstat','time','etaRaw','vesselId','portId','time_2','latitude','longitude'],axis=1)
train_full = train.copy()
train_full = train.drop(['cog','sog','rot','heading','navstat','time','etaRaw','vesselId','portId','time_2'],axis=1)
train_lon = train_full.loc[:,['longitude']]
train_lat = train_full.loc[:,['latitude']]
train_full = train_full.drop(['latitude','longitude'],axis=1)

In [15]:
forest_lon = RandomForestRegressor(n_estimators=100, random_state=42)
forest_lon.fit(train_full, train_lon.values.ravel())
forest_lat = RandomForestRegressor(n_estimators=100, random_state=42)
forest_lat.fit(train_full, train_lat.values.ravel())
longitude_predicted = forest_lon.predict(test_full)
latitude_predicted = forest_lat.predict(test_full)


forest_prediction = pd.DataFrame({'longitude_predicted':longitude_predicted,'latitude_predicted':latitude_predicted})
forest_prediction = forest_prediction.reset_index()
forest_prediction.rename(columns={'index': 'ID'}, inplace=True)
forest_prediction.to_csv('data/forest_prediction_extra.csv', index=False)

KeyboardInterrupt: 